In [ ]:
import pandas as pd 
import numpy as np
from google.colab import data_table
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold

In [ ]:
# a function for visualizing data by using google colab data_table tool
def visual(inp):
  data_table.DataTable.max_columns=100
  return data_table.DataTable(inp)

In [ ]:
# import the data from a csv file
df=pd.read_csv("/content/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv",low_memory=False)
df.shape

(225745, 79)

In [ ]:
# data description
visual(df.describe())

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,225745.00000,2.257450e+05,225745.000000,225745.000000,225745.000000,2.257450e+05,225745.000000,225745.000000,225745.000000,225745.000000,...,225745.000000,225745.000000,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05
mean,8879.61946,1.624165e+07,4.874916,4.572775,939.463346,5.960477e+03,538.535693,27.882221,164.826715,214.907242,...,3.311497,21.482753,1.848261e+05,1.293436e+04,2.080849e+05,1.776201e+05,1.032214e+07,3.611943e+06,1.287813e+07,7.755355e+06
std,19754.64740,3.152437e+07,15.422874,21.755356,3249.403484,3.921834e+04,1864.128991,163.324159,504.892965,797.411073,...,12.270018,4.166799,7.979250e+05,2.102737e+05,9.002350e+05,7.842602e+05,2.185303e+07,1.275689e+07,2.692126e+07,1.983109e+07
min,0.00000,-1.000000e+00,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,80.00000,7.118000e+04,2.000000,1.000000,26.000000,0.000000e+00,6.000000,0.000000,6.000000,0.000000,...,1.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,80.00000,1.452333e+06,3.000000,4.000000,30.000000,1.640000e+02,20.000000,0.000000,8.666667,5.301991,...,2.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,80.00000,8.805237e+06,5.000000,5.000000,63.000000,1.160100e+04,34.000000,6.000000,32.000000,10.263203,...,4.000000,20.000000,1.878000e+03,0.000000e+00,1.878000e+03,1.862000e+03,8.239725e+06,0.000000e+00,8.253838e+06,7.422849e+06
max,65532.00000,1.199999e+08,1932.000000,2942.000000,183012.000000,5.172346e+06,11680.000000,1472.000000,3867.000000,6692.644993,...,1931.000000,52.000000,1.000000e+08,3.950000e+07,1.000000e+08,1.000000e+08,1.200000e+08,6.530000e+07,1.200000e+08,1.200000e+08


In [ ]:
# convert str value of label column to int
non_numeric_label={"BENIGN":0,"DDoS":1}
df[" Label"]=df[" Label"].map(non_numeric_label)
df[" Label"].value_counts(dropna=False)
# 0 represent for no attack
# 1 represents for DDoS attack

1    128027
0     97718
Name:  Label, dtype: int64

In [ ]:
# make a list by column names
columns=df.columns.tolist()

# maximum value of each column
for i in range(0,len(columns)):
  print(f"{columns[i]} -> {df[columns[i]].max()}")


 Destination Port -> 65532
 Flow Duration -> 119999937
 Total Fwd Packets -> 1932
 Total Backward Packets -> 2942
Total Length of Fwd Packets -> 183012
 Total Length of Bwd Packets -> 5172346
 Fwd Packet Length Max -> 11680
 Fwd Packet Length Min -> 1472
 Fwd Packet Length Mean -> 3867.0
 Fwd Packet Length Std -> 6692.644993
Bwd Packet Length Max -> 11680
 Bwd Packet Length Min -> 1460
 Bwd Packet Length Mean -> 5800.5
 Bwd Packet Length Std -> 8194.660487
Flow Bytes/s -> inf
 Flow Packets/s -> inf
 Flow IAT Mean -> 107000000.0
 Flow IAT Std -> 69200000.0
 Flow IAT Max -> 120000000
 Flow IAT Min -> 107000000
Fwd IAT Total -> 120000000
 Fwd IAT Mean -> 120000000.0
 Fwd IAT Std -> 76700000.0
 Fwd IAT Max -> 120000000
 Fwd IAT Min -> 120000000
Bwd IAT Total -> 120000000
 Bwd IAT Mean -> 120000000.0
 Bwd IAT Std -> 76700000.0
 Bwd IAT Max -> 120000000
 Bwd IAT Min -> 120000000
Fwd PSH Flags -> 1
 Bwd PSH Flags -> 0
 Fwd URG Flags -> 0
 Bwd URG Flags -> 0
 Fwd Header Length -> 39396
 Bwd He

In [ ]:
# put all ports and their usage in a dictonary
# key: port, value: usage
port_dicts={}
for i in range(df.shape[0]):
  port_in_the_data=df[" Destination Port"][i]
  if port_in_the_data in port_dicts:
    port_dicts[port_in_the_data]+=1
  else:
    port_dicts.update({port_in_the_data : 1})

# sort with respect to the usage of them
from collections import OrderedDict
port_dicts = OrderedDict(sorted(port_dicts.items()))

# find the most used port
values=port_dicts.values()
max_used_port=max(values)

# find the port
for i in port_dicts:
  if(port_dicts[i]==max_used_port):
    print(f"Max used port is '{i}' with '{max_used_port}' calls")

Max used port is '80' with '136951' calls


In [ ]:
# port 80 which is most used port attacked and not attacked values
print(f"port 80, used: {len(df.loc[(df[' Destination Port']==80)])}")
print(f"port 80, attacked: {len(df.loc[(df[' Destination Port']==80) & (df[' Label']==1)])}")
print(f"port 80, not attacked: {len(df.loc[(df[' Destination Port']==80) & (df[' Label']==0)])}")

port 80, used: 136951
port 80, attacked: 128024
port 80, not attacked: 8927


In [ ]:
# optimize the int type values
for i in columns:
  if (df[f"{i}"].dtype=='int64'):
    if (df[f"{i}"].max()<127):
      df[f"{i}"]=df[f"{i}"].astype('int8')
    elif(df[f"{i}"].max()<32767):
      df[f"{i}"]=df[f"{i}"].astype('int16')
    elif(df[f"{i}"].max()<2*10**9):
      df[f"{i}"]=df[f"{i}"].astype('int32')
  else:
    continue

In [ ]:
# visualize how many null values exist in data in descending
print(df.isnull().sum(axis=0).sort_values(ascending=False))

Flow Bytes/s             4
 Destination Port        0
 Bwd Avg Bytes/Bulk      0
 Fwd Avg Packets/Bulk    0
Fwd Avg Bytes/Bulk       0
                        ..
 Bwd IAT Mean            0
Bwd IAT Total            0
 Fwd IAT Min             0
 Fwd IAT Max             0
 Label                   0
Length: 79, dtype: int64


In [ ]:
# drop null values(samples) and replace to main dataset
df.dropna(subset=['Flow Bytes/s'],inplace=True)
df.dropna(subset=[' Label'],inplace=True)
df.dropna(subset=[' Init_Win_bytes_backward'],inplace=True)
df.dropna(subset=[' Fwd Avg Packets/Bulk'],inplace=True)
df.dropna(subset=[' Fwd Avg Bulk Rate'],inplace=True)

In [ ]:
# visualize how many null values exist in data in descending
print(df.isnull().sum(axis=0).sort_values(ascending=False))

 Destination Port        0
 ECE Flag Count          0
 Fwd Avg Packets/Bulk    0
Fwd Avg Bytes/Bulk       0
 Fwd Header Length.1     0
                        ..
Bwd IAT Total            0
 Fwd IAT Min             0
 Fwd IAT Max             0
 Fwd IAT Std             0
 Label                   0
Length: 79, dtype: int64


In [ ]:
visual(df.describe())
# Flow Bytes/s and Flow Packets/s max values are infinity

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
count,225741.000000,2.257410e+05,225741.000000,225741.000000,225741.000000,2.257410e+05,225741.000000,225741.000000,225741.000000,225741.000000,...,225741.000000,2.257410e+05,2.257410e+05,2.257410e+05,2.257410e+05,2.257410e+05,2.257410e+05,2.257410e+05,2.257410e+05,225741.000000
mean,8879.294213,1.624194e+07,4.874972,4.572851,939.479993,5.960583e+03,538.545235,27.882715,164.829636,214.911050,...,21.482513,1.848294e+05,1.293459e+04,2.080886e+05,1.776232e+05,1.032233e+07,3.612007e+06,1.287836e+07,7.755493e+06,0.567141
std,19754.491905,3.152458e+07,15.423004,21.755541,3249.429866,3.921868e+04,1864.144128,163.325564,504.896961,797.417625,...,4.166390,7.979317e+05,2.102755e+05,9.002425e+05,7.842668e+05,2.185318e+07,1.275700e+07,2.692145e+07,1.983124e+07,0.495473
min,0.000000,-1.000000e+00,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,80.000000,7.118300e+04,2.000000,1.000000,26.000000,0.000000e+00,6.000000,0.000000,6.000000,0.000000,...,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
50%,80.000000,1.452362e+06,3.000000,4.000000,30.000000,1.640000e+02,20.000000,0.000000,8.666667,5.301991,...,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000
75%,80.000000,8.805245e+06,5.000000,5.000000,63.000000,1.160100e+04,34.000000,6.000000,32.000000,10.263203,...,20.000000,1.878000e+03,0.000000e+00,1.878000e+03,1.862000e+03,8.239754e+06,0.000000e+00,8.253869e+06,7.422871e+06,1.000000
max,65532.000000,1.199999e+08,1932.000000,2942.000000,183012.000000,5.172346e+06,11680.000000,1472.000000,3867.000000,6692.644993,...,52.000000,1.000000e+08,3.950000e+07,1.000000e+08,1.000000e+08,1.200000e+08,6.530000e+07,1.200000e+08,1.200000e+08,1.000000


In [ ]:
# delete both of them from features
df=df.drop('Flow Bytes/s',axis='columns')
df=df.drop(' Flow Packets/s',axis='columns')

# update columns list
columns=df.columns.tolist()

In [ ]:
# other than ports column, normalize all featuers between zero and one
scaler = MinMaxScaler(feature_range=(0, 1))
for i in columns:
  try:
    if (i== ' Destination Port'):
      continue
    else:
      df[[f"{i}"]] = scaler.fit_transform(df[[f"{i}"]])
  except ValueError:
    continue

In [ ]:
# delete some columns which all of their values are zero
for k in range(len(columns)):
  if(df[columns[k]].min()==0 and df[columns[k]].max()==0):
    df=df.drop(columns[k],axis='columns')

In [ ]:
visual(df.describe())

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
count,225741.000000,225741.000000,225741.000000,225741.000000,225741.000000,225741.000000,225741.000000,225741.000000,225741.000000,225741.000000,...,225741.000000,225741.000000,225741.000000,225741.000000,225741.000000,225741.000000,225741.000000,225741.000000,225741.000000,225741.000000
mean,8879.294213,0.135350,0.002007,0.001554,0.005133,0.001152,0.046108,0.018942,0.042625,0.032112,...,0.413125,0.001848,0.000327,0.002081,0.001776,0.086019,0.055314,0.107320,0.064629,0.567141
std,19754.491905,0.262705,0.007987,0.007395,0.017755,0.007582,0.159601,0.110955,0.130566,0.119148,...,0.080123,0.007979,0.005323,0.009002,0.007843,0.182110,0.195360,0.224345,0.165260,0.495473
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,80.000000,0.000593,0.000518,0.000340,0.000142,0.000000,0.000514,0.000000,0.001552,0.000000,...,0.384615,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,80.000000,0.012103,0.001036,0.001360,0.000164,0.000032,0.001712,0.000000,0.002241,0.000792,...,0.384615,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,80.000000,0.073377,0.002071,0.001700,0.000344,0.002243,0.002911,0.004076,0.008275,0.001534,...,0.384615,0.000019,0.000000,0.000019,0.000019,0.068665,0.000000,0.068782,0.061857,1.000000
max,65532.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# data shape after pre-processing part
df.shape

(225741, 67)

In [ ]:
# save preprocessed dataset as a csv file
df.to_csv('pre_processed_dataset.csv')